# Batcher-Banyan network

In [2]:
import numpy as np

### batcher networkの実装

In [3]:
#2*2 bitnic sorter
#２つを比べて大きい方を下にする
def sorter_base(x):
    if x[0] >= x[1]:
        return x[::-1]
    else:
        return x

In [4]:
#n*n bitnic sorter
# n*n bitnic sorterは一回り小さいbitnic sorterの組み合わせで表現できるので，　任意の大きさのbitnic sorterは再帰的に表現することができる．
def sorter(x):
    n = int(len(x))
    n2 = int(n/2)
    if n == 2:
        return sorter_base(x) #2*2が最小単位
    else:
        x1 = np.zeros(n)
        #まず2*2でsort
        for i in range(n2): 
            x1[2*i:2*i+2] = sorter_base(x[2*i:2*i+2]) 
        
        x2 = np.zeros(n)
        #一回り小さいsorterに入力するために並び替える
        for i in range(n):
            if i<n2 and i%2 == 1:
                x2[i] = x1[n2+i-1]
            elif i>=n2 and i%2 == 0:
                x2[i] = x1[i-n2+1]
            else:
                x2[i] = x1[i]
        
        x3 = np.zeros(n)
        #上半分と下半分をそれぞれ一回り小さいbitnic sorterに入力する(再帰)
        x3[:n2] = sorter(x2[:n2])
        x3[n2:] = sorter(x2[n2:])
        return x3

In [5]:
#n batcher network
# batcher networkは2*2 bitnic sorterから初めて徐々に大きいsorterへ入力することを繰り返して実現される．
# よって入力数に等しいbitnic sorterがくるまで，　sorterに入力→並び替え→より大きいsorterに入力．．．を繰り返す．
# 任意のn*n bitnic sorterは上で定義されているのでそれを利用する．
# k: sorterの入力次元数
#p: sorterの数=n/k

def batcher(x):
    data = x
    n = len(x)
    k = 2 
    #bitnic sorterの大きさが入力と一致するまで繰り返す．
    while k <= n:
        p = int(n/k) 
        
        data1 = np.zeros(n)
        #まず入力を適切に並び替える．
        for i in range(p): 
            for j in range(k):
                if j<k/2:
                    data1[2*j+i*k] = data[j+i*k]
                else:
                    data1[2*j-k+1+i*k] = data[j+i*k]                
        
        data2 = np.zeros(n)
        #並び替えたものをそれぞれsorterに入力する．
        for i in range(p):
            s = sorter(data1[k*i:k*i+k])
            if i%2 == 1:
                s = s[::-1] #奇数番目のものは出力を反転させる
            data2[k*i:k*i+k] = s
            
        k = k*2
        data = data2
        
    return data

### banyan networkの実装

In [6]:
#2bit banyan sorter
# Noneがきたらスルー
# n_time: 判定に使うビットが先頭から何番目であるかを指定
# n_time番目のbitが0なら上，　1なら下に出力

def banyan_base(x,n_time):
    if x[0] is not None:
        if x[0][n_time] == '1':
            return x[::-1]
    elif x[1] is not None:
        if x[1][n_time] == '0':
            return x[::-1]
    
    return x

In [7]:
#n入力 banyan network
# 2bit banyan sorterを組み合わせることで実現
#内部の結び方はn*n bitnic sorterと同様になるのでこちらも再帰的に表現できる．

def banyan(x,n_time):
    n = int(len(x))
    n2 = int(n/2)
    
    if n == 2: #2*2が最小単位
        return banyan_base(x,n_time)
    else:
        x1 = [''] * n
        #まず2*2でsort
        for i in range(n2): 
            x1[2*i:2*i+2] = banyan_base(x[2*i:2*i+2],n_time)
        
        x2 = [''] * n
        #一回り小さいsorterに入力するために並び替える
        for i in range(n): 
            if i<n2 and i%2 == 1:
                x2[i] = x1[n2+i-1]
            elif i>=n2 and i%2 == 0:
                x2[i] = x1[i-n2+1]
            else:
                x2[i] = x1[i]
        
        x3 = [''] * n 
        #上半分と下半分をそれぞれ一回り小さいbitnic sorterに入力する(再帰)
        x3[:n2] = banyan(x2[:n2],n_time+1) #判定に使うbitを何番目にするかについて，　値を1増やす(n_time+1)
        x3[n2:] = banyan(x2[n2:],n_time+1)
        return x3
    

### 実行プログラム 

In [9]:
#入力数列
#x = np.array([1,1000,4,6,1000,0,1000,7]) #8要素
#x = np.array([13,1000,4,6,1000,11,1000,8,1,1000,15,1000,7,3,9,1000]) #16要素
x = np.array([13,1000,29,6,1000,11,1000,8,25,1000,15,1000,7,28,19,1000,
             1000,31,1000,1000,27,18,4,1,1000,1000,3,21,16,1000,9,1000]) #32要素

print('---input--- \n', x)
n = int(len(x))
n_time = int(np.log2(n))
print('---x size--- \n', n)

# batcher network
x1 = batcher(x)
print('---batcher output--- \n', x1)

# batcher networkの出力を banyan network　に入力するために並び替える
x2 = np.zeros(n)
for i in range(n):
    if i<n/2:
        x2[2*i] = x1[i]
    else:
        x2[2*i-n+1] = x1[i]

# 10進数から2進数表現に変更
x3 = [None]*n
for i in range(n):
    if x2[i] < n:
        x3[i] = format(int(x2[i]), '0{}b'.format(n_time)) #n bitの2進数表現
print('---banyan input--- \n', x3)

#banyan network
x4 = banyan(x3,0)
print('---banyan output--- \n', x4)

# 確認のためbanyan networkの出力を10進数に変更
x5 = [None] * n
for i in range(n):
    if x4[i] != None:
        x5[i] = int(x4[i], 2)
print('---final output--- \n', x5)

---input--- 
 [  13 1000   29    6 1000   11 1000    8   25 1000   15 1000    7   28
   19 1000 1000   31 1000 1000   27   18    4    1 1000 1000    3   21
   16 1000    9 1000]
---x size--- 
 32
---batcher output--- 
 [   1.    3.    4.    6.    7.    8.    9.   11.   13.   15.   16.   18.
   19.   21.   25.   27.   28.   29.   31. 1000. 1000. 1000. 1000. 1000.
 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000.]
---banyan input--- 
 ['00001', '11100', '00011', '11101', '00100', '11111', '00110', None, '00111', None, '01000', None, '01001', None, '01011', None, '01101', None, '01111', None, '10000', None, '10010', None, '10011', None, '10101', None, '11001', None, '11011', None]
---banyan output--- 
 [None, '00001', None, '00011', '00100', None, '00110', '00111', '01000', '01001', None, '01011', None, '01101', None, '01111', '10000', None, '10010', '10011', None, '10101', None, None, None, '11001', None, '11011', '11100', '11101', None, '11111']
---final output--- 
 [None, 1, None, 3, 4,